In [1]:
import requests
import pandas as pd
import datetime
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

## Fetching URL and Normalizing data into pandas DF

In [4]:
def getJSONdata(url):
    response = requests.get(url)
    dat = pd.json_normalize(response.json())
    return dat

In [6]:
booster_dat = getJSONdata("https://api.spacexdata.com/v4/rockets/")
launch_dat = getJSONdata("https://api.spacexdata.com/v4/launchpads/")
payload_dat = getJSONdata("https://api.spacexdata.com/v4/payloads/")
data = getJSONdata("https://api.spacexdata.com/v4/launches/past")

In [7]:
# Takes the dataset and uses the cores column to call the API and append the data to the lists
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])

In [10]:
# Lets take a subset of our dataframe keeping only the features we want and the flight number, and date_utc.
data = data[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

# We will remove rows with multiple cores because those are falcon rockets with 2 extra rocket boosters and rows that have multiple payloads in a single rocket.
data = data[data['cores'].map(len)==1]
data = data[data['payloads'].map(len)==1]

# Since payloads and cores are lists of size 1 we will also extract the single value in the list and replace the feature.
data['cores'] = data['cores'].map(lambda x : x[0])
data['payloads'] = data['payloads'].map(lambda x : x[0])

# We also want to convert the date_utc to a datetime datatype and then extracting the date leaving the time
data['date'] = pd.to_datetime(data['date_utc']).dt.date

# Using the date we will restrict the dates of the launches
data = data[data['date'] <= datetime.date(2021, 11, 13)]
data = pd.DataFrame(data)
data.head()

,rocket,payloads,launchpad,cores,flight_number,date_utc,date
0,5e9d0d95eda69955f709d1eb,5eb0e4b5b6c3bb0006eeb1e1,5e9e4502f5090995de566f86,"{'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",1,2006-03-24T22:30:00.000Z,2006-03-24
1,5e9d0d95eda69955f709d1eb,5eb0e4b6b6c3bb0006eeb1e2,5e9e4502f5090995de566f86,"{'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",2,2007-03-21T01:10:00.000Z,2007-03-21
3,5e9d0d95eda69955f709d1eb,5eb0e4b7b6c3bb0006eeb1e5,5e9e4502f5090995de566f86,"{'core': '5e9e289ef3591855dc3b2626', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",4,2008-09-28T23:15:00.000Z,2008-09-28
4,5e9d0d95eda69955f709d1eb,5eb0e4b7b6c3bb0006eeb1e6,5e9e4502f5090995de566f86,"{'core': '5e9e289ef359184f103b2627', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",5,2009-07-13T03:35:00.000Z,2009-07-13
5,5e9d0d95eda69973a809d1ec,5eb0e4b7b6c3bb0006eeb1e7,5e9e4501f509094ba4566f84,"{'core': '5e9e289ef359185f2b3b2628', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",6,2010-06-04T18:45:00.000Z,2010-06-04


## Subsetting the data sets to make them smaller for project. Including attributes that I think would be interesting to do analysis on

In [13]:
booster_dat_sub = booster_dat[['id','name','cost_per_launch','success_rate_pct','engines.type','engines.layout','engines.number',]]
booster_dat_sub.rename(columns={"id":"rocket_id","name":"booster_name",'engines.type':'engine_type','engines.layout':'engine_layout','engines.number' : 'engines_number' },inplace=True)
booster_dat_sub

C:\Users\banto\AppData\Local\Temp\ipykernel_21208\148631751.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  booster_dat_sub.rename(columns={"id":"rocket_id","name":"booster_name",'engines.type':'engine_type','engines.layout':'engine_layout','engines.number' : 'engines_number' },inplace=True)


,rocket_id,booster_name,cost_per_launch,success_rate_pct,engine_type,engine_layout,engines_number
0,5e9d0d95eda69955f709d1eb,Falcon 1,6700000,40,merlin,single,1
1,5e9d0d95eda69973a809d1ec,Falcon 9,50000000,98,merlin,octaweb,9
2,5e9d0d95eda69974db09d1ed,Falcon Heavy,90000000,100,merlin,octaweb,27
3,5e9d0d96eda699382d09d1ee,Starship,7000000,0,raptor,None,37


In [15]:
launch_dat_sub = launch_dat[['id', 'name','region','longitude','latitude']]
launch_dat_sub.rename(columns={"id":"launchpad_id",'name':'launchpad_name'},inplace=True)
launch_dat_sub

C:\Users\banto\AppData\Local\Temp\ipykernel_21208\1796231168.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  launch_dat_sub.rename(columns={"id":"launchpad_id",'name':'launchpad_name'},inplace=True)


,launchpad_id,launchpad_name,region,longitude,latitude
0,5e9e4501f5090910d4566f83,VAFB SLC 3W,California,-120.593144,34.644090
1,5e9e4501f509094ba4566f84,CCSFS SLC 40,Florida,-80.577366,28.561857
2,5e9e4502f5090927f8566f85,STLS,Texas,-97.156085,25.997264
3,5e9e4502f5090995de566f86,Kwajalein Atoll,Marshall Islands,167.743129,9.047721
4,5e9e4502f509092b78566f87,VAFB SLC 4E,California,-120.610829,34.632093
5,5e9e4502f509094188566f88,KSC LC 39A,Florida,-80.603956,28.608058


In [17]:
payload_dat_sub = payload_dat[['id','launch', 'orbit','mass_lbs']]
payload_dat_sub.rename(columns={"id":"payload_id",'launch':'launchpad_id'},inplace=True)
payload_dat_sub

C:\Users\banto\AppData\Local\Temp\ipykernel_21208\1987054697.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  payload_dat_sub.rename(columns={"id":"payload_id",'launch':'launchpad_id'},inplace=True)


,payload_id,launchpad_id,orbit,mass_lbs
0,5eb0e4b5b6c3bb0006eeb1e1,5eb87cd9ffd86e000604b32a,LEO,43.0
1,5eb0e4b6b6c3bb0006eeb1e2,5eb87cdaffd86e000604b32b,LEO,NaN
2,5eb0e4b6b6c3bb0006eeb1e3,5eb87cdbffd86e000604b32c,LEO,NaN
3,5eb0e4b6b6c3bb0006eeb1e4,5eb87cdbffd86e000604b32c,LEO,NaN
4,5eb0e4b7b6c3bb0006eeb1e5,5eb87cdbffd86e000604b32d,LEO,363.0
...,...,...,...,...
220,631616a7ffc78f3b8567071a,63161339ffc78f3b8567070c,VLEO,29233.0
221,631616aeffc78f3b8567071b,62f3b5200f55c50e192a4e6c,VLEO,29233.0
222,63161760ffc78f3b8567071c,62f3b5200f55c50e192a4e6c,PO,NaN
223,631617fbffc78f3b8567071d,62f3b5330f55c50e192a4e6e,VLEO,396.0


In [19]:
data_sub = data[['rocket', 'payloads', 'launchpad', 'flight_number', 'date']]
data_sub.rename(columns={"rocket":"rocket_id",'payloads':'payload_id','launchpad':'launchpad_id'},inplace=True)
data_sub

C:\Users\banto\AppData\Local\Temp\ipykernel_21208\3375035713.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sub.rename(columns={"rocket":"rocket_id",'payloads':'payload_id','launchpad':'launchpad_id'},inplace=True)


,rocket_id,payload_id,launchpad_id,flight_number,date
0,5e9d0d95eda69955f709d1eb,5eb0e4b5b6c3bb0006eeb1e1,5e9e4502f5090995de566f86,1,2006-03-24
1,5e9d0d95eda69955f709d1eb,5eb0e4b6b6c3bb0006eeb1e2,5e9e4502f5090995de566f86,2,2007-03-21
3,5e9d0d95eda69955f709d1eb,5eb0e4b7b6c3bb0006eeb1e5,5e9e4502f5090995de566f86,4,2008-09-28
4,5e9d0d95eda69955f709d1eb,5eb0e4b7b6c3bb0006eeb1e6,5e9e4502f5090995de566f86,5,2009-07-13
5,5e9d0d95eda69973a809d1ec,5eb0e4b7b6c3bb0006eeb1e7,5e9e4501f509094ba4566f84,6,2010-06-04
...,...,...,...,...,...
132,5e9d0d95eda69973a809d1ec,5fe3c4f2b3467846b3242193,5e9e4502f509094188566f88,133,2021-08-29
133,5e9d0d95eda69973a809d1ec,60e3bf3373359e1e20335c3c,5e9e4502f509092b78566f87,134,2021-09-14
134,5e9d0d95eda69973a809d1ec,607a382f5a906a44023e0867,5e9e4502f509094188566f88,135,2021-09-16
135,5e9d0d95eda69973a809d1ec,5fe3b3bab3467846b3242174,5e9e4502f509094188566f88,136,2021-11-11


## Writing SQL DDL to insert tables into Database & inserting data into tables

In [22]:
sql_statements = [
    
"""
CREATE TABLE IF NOT EXISTS booster_version (
rocket_id VARCHAR (255) PRIMARY KEY,
cost_per_launch INT,
success_rate_pct INT,
engines_type VARCHAR (255),
engines_layout VARCHAR (255),
engines_number INT,
name VARCHAR (255)
);

""",

"""CREATE INDEX idx_booster ON booster_version(rocket_id);""",



"""
CREATE TABLE IF NOT EXISTS launch_site (
launchpad_id VARCHAR (255) PRIMARY KEY,
rocket_id VARCHAR (255),
region VARCHAR (255),
longitude   INT,
latitude   INT,
name VARCHAR (255),
FOREIGN KEY(rocket_id) REFERENCES booster_version(rocket_id)
);

""",

"""CREATE INDEX idx_launch_site ON launch_site(launchpad_id,rocket_id);""",



"""
CREATE TABLE IF NOT EXISTS payload (
payload_id VARCHAR (255) PRIMARY KEY,
payload_mass INT
orbit VARCHAR (255)
);

""",

"""CREATE INDEX idx_payload ON payload(payload_id);""",




"""CREATE TABLE IF NOT EXISTS core_data (
rocket_id VARCHAR (255) PRIMARY KEY,
payload_id VARCHAR (255),
launchpad_id VARCHAR (255),
cores VARCHAR (255),
flightnumber INT,
date_utc DATE,
date DATE, 
FOREIGN KEY(rocket_id) REFERENCES booster_version(rocket_id)
FOREIGN KEY(payload_id) REFERENCES payload(payload_id)
FOREIGN KEY(launchpad_id) REFERENCES launch_site(launchpad_id)
);

""",

"""CREATE INDEX idx_core_data ON core_data (rocket_id,payload_id,launchpad_id);"""




]

In [24]:
import sqlite3
# create a database connection
try:
    with sqlite3.connect('bbanton_interview.db') as conn:
        # create a cursor
        cursor = conn.cursor()

        # execute statements
        for statement in sql_statements:
            cursor.execute(statement)

        # commit the changes
        conn.commit()

        print("Tables created successfully.")
except sqlite3.OperationalError as e:
    print("Failed to create tables:", e)

Tables created successfully.


In [26]:
booster_dat_sub.to_sql(name = 'booster_version', con = conn, if_exists = 'replace', index = False)
conn.commit()

In [28]:
launch_dat_sub.to_sql(name = 'launch_site', con = conn, if_exists = 'replace', index = False)
conn.commit()

In [30]:
payload_dat_sub.to_sql(name = 'payload', con = conn, if_exists = 'replace', index = False)
conn.commit()

In [32]:
data_sub.to_sql(name = 'core_data', con = conn, if_exists = 'replace', index = False)
conn.commit()

### Joining the 'base' tables togeter into a final table called launch_data and writing DDL to create table in DB

In [35]:
launch_data = pd.merge(data_sub, booster_dat_sub, on = 'rocket_id')\
                .merge(launch_dat_sub, on = 'launchpad_id')\
                .merge(payload_dat_sub, on = 'payload_id')

In [37]:
launch_data_sub = launch_data.drop('launchpad_id_y',axis = 1)
launch_data_sub.rename(columns={"launchpad_id_x":"launchpad_id"},inplace=True)

In [39]:
sql_statements2 = [
    
"""
CREATE TABLE IF NOT EXISTS launch_data (
    rocket_id        VARCHAR (255) PRIMARY KEY,
    payload_id       VARCHAR (255),
    launchpad_id     VARCHAR (255),
    flightnumber     INT,
    date             DATE,
    booster_name     VARCHAR (255),
    cost_per_launch  INT,
    success_rate_pct INT,
    engine_type      VARCHAR (255),
    engine_layout    VARCHAR (255),
    engines_number   INT,
    launchpad_name   VARCHAR (255),
    region           VARCHAR (255),
    longitude        INT,
    latitude         INT,
    orbit            VARCHAR (255),
    mass_lbs         VARCHAR (255),
FOREIGN KEY(rocket_id) REFERENCES booster_version(rocket_id)
FOREIGN KEY(payload_id) REFERENCES payload(payload_id)
FOREIGN KEY(launchpad_id) REFERENCES launch_site(launchpad_id)
FOREIGN KEY(rocket_id) REFERENCES core_data(rocket_id)
FOREIGN KEY(payload_id) REFERENCES core_data(payload_id)
FOREIGN KEY(launchpad_id) REFERENCES core_data(launchpad_id)
);

""",

"""CREATE INDEX idx_launch_data ON launch_data(rocket_id,payload_id,launchpad_id);"""
]

#### To view this table you will need to refresh the database to see the new launch_data table

In [41]:
launch_data_sub.to_sql(name = 'launch_data', con = conn, if_exists = 'replace', index = False)
conn.commit()